In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas
areaurl = 'http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index701.html'
res = requests.get(areaurl)
driver = webdriver.Chrome('/Applications/chromedriver')
driver.get(areaurl)
soup = BeautifulSoup(driver.page_source, 'lxml')
links = soup.select('div[class^=main-item] a')
domain = 'http://vote.2018.nat.gov.tw/pc/zh_TW'
area_vote = [domain + ele.get('href').strip('.') for ele in links]
for ele in area_vote:
    print(ele)

http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index100.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index200.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index300.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index400.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index500.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index600.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index701.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index702.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index703.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index704.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index705.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index706.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index707.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index708.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index709.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index710.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index711.html
http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index801.html
http://vot

In [2]:
def getRef(url):
    res = requests.get(url)
    dfs = pandas.read_html(res.text)
    soup = BeautifulSoup(res.text, 'lxml')
    votes = dfs[1]
    votes.columns = votes.loc[1]
    votes.drop([0,1], inplace=True)
    drop = [ '投開票所' in i for i in votes['號次'] ]
    drop = drop.index(True) +2
    votes.drop([drop], inplace = True)
    soup = BeautifulSoup(res.text, 'lxml')
    area = soup.select_one('b').text
    aname_pos = area.find('區')
    if (aname_pos == -1):
        aname_pos = area.find('鄉')
    if (aname_pos == -1):
        aname_pos = area.find('鎮')
    if (aname_pos == -1):
        aname_pos = area.find('市')
    aname_pos_start = area.index('在')
    aname = area[0:3] + ' ' + area[aname_pos_start +1 : aname_pos+1]
    votes['地區'] = aname
    return (votes)

In [3]:
def write_votecsv(area_url):
    driver.get(area_url)
    domain = 'http://vote.2018.nat.gov.tw/pc/zh_TW'
    results = []
    soup = BeautifulSoup(driver.page_source, 'lxml')
    links = soup.select('div[id^=item] a')
    #TC 是市長
    #PV 是里長
    data_select = ["TC" in ele.get('href').strip('.') for ele in links] #only get data of mayor
    getdata = []
    for i in range(len(links)):
        if (data_select[i] == True):
            getdata.append(domain + links[i].get('href').strip('.'))
    for ele in getdata:
        try:
            results.append(getRef(ele))
        except:
            print("cannot get data from", ele)
    new_results = pandas.concat(results)
    res = requests.get(getdata[0])
    soup = BeautifulSoup(res.text, 'lxml')
    area = soup.select_one('b').text
    pos = area.index('\xa0')
    fname = area[0:pos]
    fname += '.csv'
    new_results.to_csv(fname, sep='\t', encoding='utf-8')

In [4]:
getRef('http://vote.2018.nat.gov.tw/pc/zh_TW/TC/nm10004000100000000.html')

1,註記,號次,姓名,性別,得票數,得票率,推薦之政黨,地區
2,◎,1,楊文科,男,27664,33.50%,中國國民黨,新竹縣 竹北市
3,NaN,2,葉芳棟,男,1901,2.30%,無,新竹縣 竹北市
4,NaN,3,鄭朝方,男,21725,26.31%,民主進步黨,新竹縣 竹北市
5,NaN,4,徐欣瑩,女,31292,37.89%,民國黨,新竹縣 竹北市


In [5]:
driver.get('http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index713.html')
domain = 'http://vote.2018.nat.gov.tw/pc/zh_TW'
results = []
soup = BeautifulSoup(driver.page_source, 'lxml')
links = soup.select('div[id^=item] a')
#TC 是市長
#PV 是里長
data_select = ["TC" in ele.get('href').strip('.') for ele in links] #only get data of mayor
getdata = []
for i in range(len(links)):
    if (data_select[i] == True):
        getdata.append(domain + links[i].get('href').strip('.'))
for ele in getdata:
    try:
        results.append(getRef(ele))
    except:
        print("cannot get data from", ele)
new_results = pandas.concat(results)
res = requests.get(getdata[0])
soup = BeautifulSoup(res.text, 'lxml')
area = soup.select_one('b').text
pos = area.index('\xa0')
fname = area[0:pos]
fname += '.csv'
new_results

1,註記,號次,姓名,性別,得票數,得票率,推薦之政黨,地區
2,NaN,1,謝文進,男,19364,19.45%,無,新竹市 東區
3,NaN,2,李驥羣,男,323,0.32%,無,新竹市 東區
4,NaN,3,黃源甫,男,2165,2.17%,無,新竹市 東區
5,NaN,4,許明財,男,29097,29.23%,中國國民黨,新竹市 東區
6,NaN,5,郭榮睿,男,261,0.26%,無,新竹市 東區
7,◎,6,林智堅,男,48346,48.56%,民主進步黨,新竹市 東區
2,NaN,1,謝文進,男,17017,22.39%,無,新竹市 北區
3,NaN,2,李驥羣,男,236,0.31%,無,新竹市 北區
4,NaN,3,黃源甫,男,1053,1.39%,無,新竹市 北區
5,NaN,4,許明財,男,20850,27.43%,中國國民黨,新竹市 北區


In [6]:
res = requests.get('http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index200.html')
driver = webdriver.Chrome('/Applications/chromedriver')
driver.get('http://vote.2018.nat.gov.tw/pc/zh_TW/IDX/index200.html')
soup = BeautifulSoup(driver.page_source, 'lxml')
links = soup.select('div[class^=main-item] a')
domain = 'http://vote.2018.nat.gov.tw/pc/zh_TW'
area_vote = [domain + ele.get('href').strip('.') for ele in links]
for ele in area_vote:
    write_votecsv(ele)